# Bayesian network creation
The use of the package is focused on the `BayesianNetwork` class. 

## Using the constructor
If you already have a graph structure and the network parameters (or joint probability distribution) in the right formats, it is posible to use the constructor for building the network.

The graph structure is represented using a `DiGraph` object from the `networkx` package.

In [ ]:
from networkx import DiGraph

graph = DiGraph()
graph.add_nodes_from([1, 2])
graph.add_edges_from([(1, 2)])

The network parameters are represented with a dictionary where the keys are the identifiers of the nodes (they must be the same as in the `DiGraph` object) and the values are `GaussianNode` objects (at the moment, the discrete case is not yet supported). `GaussianNode` is just a named tuple with four elements: `mean`, `var`, `parents` and `parents_coeffs`.

In [ ]:
from neurogenpy import GaussianNode

parameters = {1: GaussianNode(0, 1, [], []), 2: GaussianNode(0, 1, [1], [0.8])}

Once you have both the `graph` and `parameters`, the network can be instantiated the usual way.

In [ ]:
from neurogenpy import BayesianNetwork

bn = BayesianNetwork(graph=graph, parameters=parameters)

## Learning the full network from data
Learning the structure and parameters of a Bayesian network from the data in a CSV file.

First of all, you should create a `pandas DataFrame` from your data with the structure:

| Instances  |    Feature 1 |    Feature 2 | ... |    Feature N |
|------------|:------------:|:------------:|:---:|:------------:|
| Instance 1 | $Value_{11}$ | $Value_{12}$ | ... | $Value_{1N}$ |
| Instance 2 | $Value_{21}$ | $Value_{22}$ | ... | $Value_{2N}$ |
| ...        |     ...      |       ...    | ... |       ...    |
| Instance n | $Value_{n1}$ | $Value_{n2}$ | ... | $Value_{nN}$ |

In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')

### Set the structure and parameter learning methods using arguments
Once you have read the file, you can fit it using the `fit` method and setting the structure learning algorithm, estimation method.

In [ ]:
bn = BayesianNetwork().fit(df, data_type='continuous', estimation='mle', algorithm='pc')

Additional parameters for the structure learning or parameters estimation algorithm can be provided too.

In [ ]:
bn = BayesianNetwork().fit(df, data_type='continuous', estimation='mle', algorithm='pc', penalty=0.01)

### Instance a particular `LearnStructure` or `LearnParameters` subclass
Another option is to use the desired subclass of `LearnStructure` or `LearnParameters`.

In [ ]:
from neurogenpy import PC, GaussianMLE

pc = PC(df, data_type='continuous')

mle = GaussianMLE(df)

bn = BayesianNetwork().fit(algorithm=pc, estimation=mle)

## Combinations
You can use combinations of the above methods to build your network.

If you are only interested in the graph structure, it is possible to just learn the structure and not the parameters by not providing any value for the attribute `estimation`.

In [ ]:
bn = BayesianNetwork().fit(df, data_type='continuous', algorithm='pc')

On the other hand, if you already have a graph structure and want to learn the parameters, you can provide it in the constructor or load it before calling `fit` with `skip_structure` set to `True`.

In [ ]:
bn = BayesianNetwork(graph=graph)
bn.fit(df, data_type='continuous', estimation='mle')

bn = BayesianNetwork().load('adjacency_matrix.csv')
bn.fit(df, data_type='continuous', estimation='mle')